<a href="https://colab.research.google.com/github/AjaySharvesh3/Carrer-Prediction/blob/master/Alzheimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np
from keras.applications import vgg16
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_data_dir = '/content/drive/My Drive/Colab Notebooks/Train/Train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/Test/'

In [4]:
batch_size = 16
img_height = 208
img_width = 176
numClasses=4
epoch=10

train_datagen= ImageDataGenerator(
        rescale= 1./255, 
        shear_range= 0.2,
        zoom_range= 0.2, 
        horizontal_flip= True)

validation_datagen= ImageDataGenerator(
        rescale= 1./255)

train_generator= train_datagen.flow_from_directory(
        train_data_dir, target_size= (img_height, img_width),
        batch_size= batch_size,
        class_mode='categorical')

validation_generator= validation_datagen.flow_from_directory(
        validation_data_dir, target_size= (img_height, img_width),
        batch_size= batch_size,
        class_mode= 'categorical')

Found 5121 images belonging to 4 classes.
Found 841 images belonging to 4 classes.


In [0]:
base_model= keras.applications.vgg16.VGG16(weights= 'imagenet', include_top= False, input_shape=(img_height, img_width, 3
                                                                                                ))
#print(vgg_model.summary())
#print(type(vgg_model))

In [7]:
add_model= Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation= 'relu'))
add_model.add(Dense(4, activation='softmax'))

model= Model(inputs= base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

model.fit_generator(train_generator,
        epochs=epoch,
        validation_data=validation_generator,
        verbose=1,	
 	steps_per_epoch=steps_per_epoch,
	validation_steps=validation_steps
        )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 208, 176, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 208, 176, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 208, 176, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 104, 88, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 104, 88, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 104, 88, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 52, 44, 128)       0         
__________

In [0]:
accuracies = cross_val_score(estimator = add_model, X = X_train, y = y_train, cv = 30)
max = accuracies.max()